# Context managers and the `with` keyword

A context manager exist to control a `with` instruction in the same manner iterators exist to control a `for` instruction

The `with` was designed to simplify common case of use of `try/finally`. Some use cases:
- transactions to databases
- manipulate locks, conditions and semaphores
- personalized environments to use the `Decimal` type
- patch objects for tests

The interface to a context manager consists of the methods `__enter__` and `__exit__`. The `__enter__` function will be called with no arguments but `self`. The `__exit__` function will receive three arguments:
- exc_type: the exception that might happen inside the with block
- exc_value: the exception instance
- traceback: a traceback object

In [4]:
with open('test.txt') as fp:
    src = fp.read(60)

len(src) # 60
fp # <_io.TextIOWrapper name='test.txt' mode='r' encoding='cp1252'>
fp.closed() # True
fp.read(60) # Error: file closed

<_io.TextIOWrapper name='test.txt' mode='r' encoding='cp1252'>

In [22]:
import sys

class LookingGlass:
    def __enter__(self):
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Zero Division')
            return True # Inform that the exception was treated
    
with LookingGlass() as what:
    print("Test") # tseT
    print(what) # YKCOWREBBAJ , which is inversed

print(what) # JABBERWOCKY

manager = LookingGlass()
monster = manager.__enter__()
print(monster == 'JABBERWOCKY') # eurT
monster # YKCOWREBBAJ'
manager.__exit__(None, None, None)



tseT
YKCOWREBBAJ
JABBERWOCKY
eurT


# Context managers on the `contextlib`

The standard lib has some useful context managers available out of the box. Take a look at the book to see them

The most useful of them is the `@contextmanager` decorator that we can use to replace the implementation of the `__enter__/__exit__` functions. We can just use the decorator with the `yield` expression to what the `__enter__()` function would return.

For this decorator, the `yield` split the body of the function in two: everything before the `yield` will be executed in the beginning of the `with` block and everything after the `yield` will be executed when the `with` block ends

The example below replaces the `LookingGlass` class shown in the previous example

```python
from contextlib import contexmanager
import sys

@contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write
```

Notice that the above implementation doesn't treat any possible exception that might occur in the `with` block. To fix that we need:

```python
from contextlib import contexmanager
import sys

@contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = "Zero division"
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)
    sys.stdout.write = original_write
```

# Pattern matching for the `lis.py` example

The `lis.py` example is extensive parser of LIS code to python. I will not enter in details on this notes. Take a look at the book for it.

# The `else` block

The python `else` keyword can be used with other blocks besides `if`, such as:
- `for`: the `else` block will be executed only if the `for` loop runs until the end, i. e., if the block is not stopped by the `break`, `continue` or `return` keyword
- `while`: the `else` block will only be executed when the expression for the `while` block becomes false. Again it will not execute if the block is stopped by the `break`, `continue` or `return` keyword.
- `try`: the `else` block will only be executed if none exceptions are thrown inside the `try` block. Beware: exceptions generated in the `else` block will not be treated by the preceding `except` block.

The difference for `else` and `finally` after a `try` block is that, the `finally` block will always be executed, even if inside the `try` block there is a `return` call.

